In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.dataset import load_dataset
from mindnlp.core import ops

from mindnlp.transformers import AutoTokenizer
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "lucadiliello/bart-small"
tokenizer_name_or_path = "lucadiliello/bart-small"

max_length = 128
lr = 1e-3
num_epochs = 5
batch_size = 8

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [2]:
# creating model

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

all_param = sum(param.numel() for param in model.parameters())

trainable_params = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False 
    else:
        print(name)
        trainable_params += param.numel()  

print(f"trainable params: {trainable_params:,d} || all params: {all_param:,d} || trainable%: {100 * trainable_params / all_param}")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
BartForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
model.encoder.layers.0.self_attn.k_proj.bias
model.encoder.layers.0.self_attn.v_proj.bias
model.encoder.layers.0.self_attn.q_proj.bias
model.encoder.layers.0.self_attn.out_proj.bias
model.encoder.layers.0.self_attn_layer_norm.bias
model.encoder.layers.0.fc1.bias
model.encoder.layers.0.fc2.bias
model.encoder.layers.0.final_layer_norm.bias
model.encoder.layers.1.self_attn.k_proj.bias
model.encoder.layers.1.self_attn.v_proj.bias
model.encoder.layers.1.self_attn.q_proj.bias
model.encoder.layers.1.self_attn.out_proj.bias
model.encoder.layers.1.self_attn_layer_norm.bias
model.encoder.layers.1.fc1.bias
model.encoder.layers.1.fc2.bias
model.encoder.layers.1.final_layer_norm.bias
model.encoder.layers.2.self_attn.k_proj.bias
model.encoder.layers.2.self_attn.v_proj.bias
model.encoder.layers.2.self_attn.q_proj.bias
model.encoder.layers.2.self_attn.out_proj.bias
model.encoder.layers.2.self_attn_layer_norm.bias
model.encoder.layers.

In [3]:
mindspore.dataset.config.set_seed(0)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree",cache_dir='/home/ma-user/work/financial_phrasebank/')

In [4]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [5]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(226873:281473297281200,MainProcess):2024-12-07-14:46:16.569.89 [mindspore/dataset/engine/datasets.py:1217] Dataset is shuffled before split.


In [6]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [7]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= '`` Serving customers locally is one of the cornerstones of customer focus for us .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [9]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [10]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[    0,   250,   367 ...     1,     1,     1],
  [    0, 20420,  1295 ...     1,     1,     1],
  [    0, 43623,     9 ...     1,     1,     1],
  ...
  [    0, 46769,  5516 ...     1,     1,     1],
  [    0,   133,   458 ...     1,     1,     1],
  [    0,   133,  1799 ...     1,     1,     1]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 3], dtype=Int64, value=
 [[    0, 12516,     2],
  [    0, 33407,     2],
  [    0, 12516,     2],
  ...
  [    0, 12516,     2],
  [    0, 12516,     2],
  [    0, 12516,     2]])}

In [11]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [12]:
from mindnlp.core import value_and_grad
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = value_and_grad(forward_fn, model.trainable_params())

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        optimizer.zero_grad()
        loss = grad_fn(**batch)
        optimizer.step()
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [00:02<00:00, 12.77it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.5984) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.469006) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.24963) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.222851)


100%|██████████| 29/29 [00:02<00:00, 13.57it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.2104) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.19095) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.1875) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.17185)


100%|██████████| 29/29 [00:02<00:00, 13.93it/s]


epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.17124) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.158066) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.16069) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.149014)


100%|██████████| 29/29 [00:02<00:00, 13.94it/s]


epoch=3: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.14537) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.135725) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.14766) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.137723)


100%|██████████| 29/29 [00:02<00:00, 13.91it/s]

epoch=4: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.1282) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.120622) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.14171) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.132526)


In [13]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=84.95575221238938 % on the evaluation dataset
eval_preds[:10]=['neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive']
ground_truth[:10]=['neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive']


In [14]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[    0, 49519, 45338 ...     1,     1,     1],
  [    0,   597,  5246 ...     1,     1,     1],
  [    0,   133,  1110 ...     1,     1,     1],
  ...
  [    0,   133,   613 ...     1,     1,     1],
  [    0,   104,  3914 ...     1,     1,     1],
  [    0, 35028,  2156 ...     1,     1,     1]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 3], dtype=Int64, value=
 [[    0, 12516,     2],
  [    0, 22173,     2],
  [    0, 12516,     2],
  ...
  [    0, 12516,     2],
  [    0, 33407,     2],
  [    0, 12516,     2]])]

In [15]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))
print("input",example['sentence'])
print("true label",example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

input `` Printed fabrics and related design expertise have always been the core of Marimekko 's business and brand .
true label neutral
['neutral']
